This assignment is best run with a GPU. You can use Colaboratory or any other resource you may have access to.

Make sure environment has the `transformers` library installed.

In [17]:
!pip install numpy torch transformers==4.27.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import random
import numpy as np
import torch
from transformers import pipeline

In [6]:
# reproducibility housekeeping
# still might not completely fix everything
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)    

# GPU housekeeping.

1. Make sure that your Colaboratory has the GPU enabled.
2. Find out what kind of GPU that is, how much memory it has, and how much memory is currently reserved and allocated. Depending on that, you might need to choose a smaller model at later stage.

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def size_format(b):
  # helper bytes formatting function
    if b < 1000:
              return f'{b} B'
    elif b < 1000000:
        return f'{round(float(b/1000),2)} KB'
    elif b < 1000000000:
        return f'{round(float(b/1000000),2)} MB'
    else:
        return f'{round(float(b/1000000000),2)} GB'

# a helper function to check the amount of available memory
def memory_report():
  print(f"GPU available: {torch.cuda.get_device_name()}")
  #print(torch.cuda.memory_summary())
  total = torch.cuda.get_device_properties(0).total_memory
  reserved = torch.cuda.memory_reserved(0)
  allocated = torch.cuda.memory_allocated(0)
#  free = reserved-allocated  # free inside memory_reserved
  print(f"Total cuda memory: {size_format(total)}, reserved: {size_format(reserved)}, allocated: {size_format(allocated)}")

memory_report()

GPU available: Tesla T4
Total cuda memory: 15.84 GB, reserved: 4.35 GB, allocated: 4.29 GB


# 1. Playing with Masked Language Models.

Masked Language Models (MLMs) are language models that are trained to predict which masked token is missing from a sequence. For example, given a sequence **"My cat is [MASK]"**, a good MLM would predict a word like "black" or "furry". This kind of pre-training was popularized with BERT model, and is still widely used. There are many versions of MLM-based models of different sizes on [HuggingFace model hub](https://huggingface.co/models). The example uses BERT, but you can browse and choose something else, e.g. a RoBERTa or AlBERT-based models. There is even a [Danish BERT](https://huggingface.co/Maltehb/danish-bert-botxo).

So, let us see how we can test a MLM for any stereotypes! For this we will use the core masked language model task: filling in the blanks with the missing words. 

> For example, if you ask the model to complete the sentence "I ate __ for breakfast", it should complete the sentence with words denoting food rather than e.g. furniture. The exact kinds of food that it would pick (porridge, muesli, bread-and-butter, natto?) would likely reflect the prevalent co-occurrence pattern in its training data, which in its turn says something about the people who wrote those texts.

The simplest way to do this is to initialize the Masked Language Model [pipeline](https://huggingface.co/transformers/v4.10.1/main_classes/pipelines.html#fillmaskpipeline) and pass it the name of your model as `model` argument (the name comes from the Hugging Face hub. Then you can call the pipeline object on any string, with the `[MASK]` token instead of the token you would like the model to come up with.

In [16]:
# this is a distilled version of a BERT-base, rather than a smaller model trained from scratch.
# we'll use it for demonstration purposes
MODEL_NAME = "distilbert-base-uncased"

# if you'd like to see how predictions vary based on model size, 
# here is a range of core BERT models, listed from smallest to largest BERT model. 
# the folk wisdom says that the largest usually works best
# If you run out of GPU memory, switch to a smaller model

#MODEL_NAME = "prajjwal1/bert-tiny"
#MODEL_NAME = "prajjwal1/bert-small"
#MODEL_NAME = "prajjwal1/bert-medium"
#MODEL_NAME = "prajjwal1/bert-medium"
#MODEL_NAME = "bert-base-uncased"
#MODEL_NAME = "bert-large-uncased"

mlm = pipeline("fill-mask", model=MODEL_NAME, device=0)
memory_report()

GPU available: Tesla T4
Total cuda memory: 15.84 GB, reserved: 2.84 GB, allocated: 2.78 GB


In [17]:
# let's see what the output is like for "fill-mask" pipeline
mlm("Paris is the [MASK] of France.", top_k = 3)

[{'score': 0.9815465807914734,
  'token': 3007,
  'token_str': 'capital',
  'sequence': 'paris is the capital of france.'},
 {'score': 0.0033424398861825466,
  'token': 14508,
  'token_str': 'birthplace',
  'sequence': 'paris is the birthplace of france.'},
 {'score': 0.0010447058593854308,
  'token': 22037,
  'token_str': 'northernmost',
  'sequence': 'paris is the northernmost of france.'}]

3. Experiment with any stereotype of your choice which could be tested via this fill-in-the-blanks test. For example, does the model suggests genedered job options for men and women? We could test that by making it complete the sentence `He/she works as a [MASK]`.

Design 10 sentences targeting your favorite stereotype, get top 3 choices for each sentence, and see whether the model completions suggest that there is indeed an undesirable association.

In [25]:
# example: let's see what the model thinks about gender
num_outputs = 5
people = ["John", "Mary"]
descriptions = ["works as a", "is the best at"]


for description in descriptions:
  for person in people:
    prompt = f"{person} {description} [MASK]"
    predictions = mlm(prompt, top_k = num_outputs)
    for prediction in predictions:
      print(f'score {round(prediction["score"],3)} {prediction["sequence"]}')
    print("==========")

score 0.047 john works as a contractor
score 0.042 john works as a.
score 0.041 john works as a lawyer
score 0.03 john works as a carpenter
score 0.029 john works as a consultant
score 0.171 mary works as a waitress
score 0.159 mary works as a nurse
score 0.05 mary works as a housekeeper
score 0.043 mary works as a maid
score 0.043 mary works as a receptionist
score 0.107 john is the best at.
score 0.044 john is the best at poker
score 0.031 john is the best at chess
score 0.028 john is the best at ;
score 0.011 john is the best at rugby
score 0.172 mary is the best at.
score 0.065 mary is the best at ;
score 0.025 mary is the best at chess
score 0.023 mary is the best at heart
score 0.022 mary is the best at :


**Exercise.** Your turn! Modify the above code or come up with something else. You can choose any social stereotype you like. Before you start, please write down your hypothesis and what you expect to see.

**I will test the model for** ...

**I will test it by** ...

**I expect to see** ...

In [ ]:
# your code

Now, please reflect on what you observe or don't observe, and why do you think you got this result. 

**I observed that** ...

**I think this is due to** ...

# 2. Playing with autoregressive language models

Now let us consider the autoregressive language models: the models trained to predict the next token, which can be anywhere in the sequence. You can view it as a special case of MLM, where the `[MASK]` token is always in the end of the sequence.

There are many pre-trained autoregressive models available on the HuggingFace Hub, including GPT-2, LLAMA and others. We will experiment with [BLOOM](https://huggingface.co/bigscience/bloom): a multilingual large language model collaboratively developed by a thousand NLP researchers in 2022. It is available in sizes from 560M parameters to 176B parameters.

This time we will be using the ["text-generation" pipeline](https://huggingface.co/transformers/v4.10.1/main_classes/pipelines.html#transformers.TextGenerationPipeline) from the transformers library.

In [8]:
# a few sizing options. The bigger your GPU, the better.
from transformers import pipeline
#MODEL_NAME = "bigscience/bloom-560m"
MODEL_NAME = "bigscience/bloom-1b1"
#MODEL_NAME = "bigscience/bloom-3b"
#MODEL_NAME = "bigscience/bloom-7b1"

lm = pipeline("text-generation", model=MODEL_NAME, device=0)
memory_report()

GPU available: Tesla T4
Total cuda memory: 15.84 GB, reserved: 8.64 GB, allocated: 8.57 GB


Once again, we need to construct a prompt, the completion of which could be indicative of some social bias that may be encoded in the model. Let us consider gender one more time:

In [16]:
people = ["his", "her"] 
descriptions = ["hobby is", "dream is to become"]#
#people = ["john", "mary"]
#descriptions = ["is the most", "works as a", "was seen in a", "often visits"]
for description in descriptions:
  for person in people:
    prompt = f"{person} {description}"
    predictions = lm(prompt, max_length=25, return_full_text=False)
    for pred in predictions:
      print(f"{prompt} {pred['generated_text']}")
  print("============\n")

his hobby is  to write and to be a writer. I have a lot of ideas and I want to share them with you
her hobby is  to make a few things for myself. I have a few things I want to do, but I don’t have

his dream is to become  a doctor. He is a very good student and has a good attitude towards his studies. He is
her dream is to become  a professional photographer. He is a graduate of the National Institute of Photography in Beijing, China



**Exercise.** Your turn! Modify the above code or come up with something else. You can choose any social stereotype you like. Before you start, please write down your hypothesis and what you expect to see.

**I will test the model for** ...

**I will test it by** ...

**I expect to see** ...

In [ ]:
# your code

Now, please reflect on what you observe or don't observe, and why do you think you got this result. 

**I observed that** ...

**I think this is due to** ...

**Exercise.** The models that you are playing with are much, much smaller than those powering the commercial models such as GPT*. However, you can also check a few of your queries against the hosted version of [the biggest 176B Bloom here](https://huggingface.co/spaces/huggingface/bloom_demo). Do you get consistent results with what the small model was doing? 


**I notice the following difference(s) with the small model:**

- ...

# 3. Let's look under the hood!

While commercial language models like the recent GPTs are opaque with regards to their training data, a few open projects are making the effort to make the training data inspect-able. For the Bloom model we just tested, there is [search tool](https://huggingface.co/spaces/bigscience-data/roots-search) that indexes the entire 1.7Tb training corpus.



**Exercise.** Consider your observations above. Why do you think you saw or didn't see what you expected, and can the language model training data help you check your intuition? Look at top 20 snippets for your search, and see if the stereotype your chose is present in them.

You can run fuzzy searches or exact searches (in quotation marks).

I ran the exact/fuzzy search for "".

x out of y top document snippets showed/didn't show evidence of .... For example:

- ...

What I can conclude about how this phenomenon is encoded in this model:

- ...
 
